In [3]:
import argparse
import datetime
import time
import numpy as np
import torch.nn
from more.datasets import load_more_data, load_data_mnist
from more.network import ARPLNet, init_weights, MoreNet
import more.train as T
from more.ARPLoss import ARPLoss
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [7]:
options: dict = {
    'save_dir': 'E:/more',
    'data_dir': 'E:',
    'max_epoch': 200,
    'batch_size': 128,
    'eval_freq': 1,
    'print_freq': 50,
    'cpu': False,
    'lr': 0.01,
    'name': "osr24507"
}
known = [3, 5, 6, 1, 8]
unknown = list(set(list(range(1, 9))) - set(known))
use_gpu = torch.cuda.is_available()
if options['cpu']:
    use = False

if use_gpu:
    print("Using GPU")
else:
    print("Using CPU")
options.update({
    'use_gpu': use_gpu
})

Using GPU


In [9]:
from more.datasets import load_osr_data

train_loader, test_loader, outloader = load_osr_data(options['data_dir'], known=known, unknown=unknown,
                                                     batch_size=options['batch_size'])
options['classes_legends'] = train_loader.dataset.get_labels_name()
options['num_classes'] = len(known)

In [11]:
feat_dim = 2
net = ARPLNet(feat_dim, options['num_classes'])
options['feat_dim'] = feat_dim
options['loss'] = "ARPLoss"
criterion = ARPLoss(**options)

if use_gpu:
    net = torch.nn.DataParallel(net).cuda()
    criterion = criterion.cuda()

net.load_state_dict(torch.load(options["save_dir"] + "/parm/net-" + options['name'] + ".parm"))
criterion.load_state_dict(torch.load(options["save_dir"] + "/parm/crit-" + options['name'] + ".parm"))

<All keys matched successfully>

In [13]:
import h5py
from torchvision.transforms import ToTensor
from torch.utils.data.dataset import Dataset

path = r"E:\test-large.h5"
h5 = h5py.File(path)
g_label = h5['sampleType'][:]
g_data = h5['sampleData'][:, :, :]
h5.close()
toTensor = ToTensor()


def filter(type=None, snr=None) -> np.ndarray:
    full = (g_label[0, :] != None)
    if (type != None):
        full = full & (g_label[0, :] == type)
    if (snr != None):
        full = full & (g_label[1, :] == snr)
    # 将TrueFalse数组转换为索引
    ids = np.argwhere(full).reshape(-1)
    return ids


def getData(type, snr):
    ids = filter(type, snr).tolist()
    length = len(ids)
    return g_data[:, :, ids], g_label[0, ids].reshape(1, -1)


class TypeSet(Dataset):
    def __init__(self, type, snr):
        self.img, self.label = getData(type, snr)
        _, self.length = self.label.shape

    def __getitem__(self, idx):
        return toTensor(self.img[:, :, idx]).float(), torch.tensor(self.label[0, idx] - 1, dtype=torch.int)

    def __len__(self):
        return self.length


def getLoader(type, snr):
    set = TypeSet(type, snr)
    #print(set.__len__())
    return torch.utils.data.DataLoader(
        dataset=set, batch_size=set.__len__(), shuffle=False,
        num_workers=0, drop_last=False)

In [20]:
x = getLoader(2, 10)

In [21]:
img, label = next(iter(x))
img.shape, label.shape

C:\Users\ZGYwi\AppData\Local\Temp\ipykernel_14688\1895323469.py:36: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return toTensor(self.img[:, :, idx]).float(), torch.tensor(self.label[0, idx] - 1, dtype=torch.int)


(torch.Size([125, 1, 195, 64]), torch.Size([125]))

In [23]:
x, y = net(img, True)
logits, _ = criterion(x, y)
predictions = logits.data.max(1)[1]

tensor([3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 4, 3, 4, 3, 3, 4, 2, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3,
        3, 3, 3, 3, 3], device='cuda:0')

In [26]:
glabels = ['singlepulse', 'freqdiv', 'LFM', 'QFM', '2FSK', '4FSK', 'BPSK', 'QPSK']

In [28]:
glabels[known[3] - 1]

'singlepulse'